<font size="5.5"><u><i>Make Injections</i></u></font>

<font size="4">Script to make injections: Multiple waveforms to noise strain data.</font>
<br/>
<font size="4">Author: Manuel David Morales</font>

## 1. Library imports

In [27]:
# Data analysis
import numpy as np 
import pandas as pd

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt

# Files/folders management
import os, glob, sys, re

# To read csv files
import csv

# Garbage collector
import gc

# Object serialization
import pickle

# Toolbox functions
from Toolbox import PSD, SNR, SNR_PyCBC

## 2. Input parameters

In [28]:
# Interferometer for noise data
# ------------------------------------------------------------
detector = "V1"   # Options: "L1", "H1", "V1"
# ------------------------------------------------------------

# Class of waveforms to be injected
# ------------------------------------------------------------
waveform_class = "3"   # Options: "1", "2", "3"
# ------------------------------------------------------------

# Seed for the random selection of waveforms to be injected
# ----------------------------------------------------------------
rdm_seed = 42     #  Options: 23 for noise gps_start 1256783872
                  #           42 for noise gps_start 1257050112
# ----------------------------------------------------------------

# Time between injections (in seconds)
# ------------------------------------------------------------
dt_inj = 8   # Options: 8, 12, 16, 32, 64
# ------------------------------------------------------------

# Jitter_lim: This value defines a fluctuation range, in which each
# injection will be located on a random sample in the interval
# [inj_time - jitter_lim, inj_time + jitter_lim] (in seconds)
# ------------------------------------------------------------
jitter_lim = 0.01   # Options: 0.3, 0.1, 0.0, 1.2, etc.
# ------------------------------------------------------------

# Time segment for SNR calculation around injection (in seconds)
# ----------------------------------------------------------------
t_segment = 4   #  [ t_inj-t_segment/2 , t_inj+t_segment/2 ] 
# ----------------------------------------------------------------

# ------> Input parameters for brief exploration and debugging

# Work on a reduced noise segment?
# ----------------------------------------------------------------
reduce_segment = 0   #  1: yes | 0: no
# ----------------------------------------------------------------

# Length of the reduced noise segment (in seconds)
# ----------------------------------------------------------------
reduced_time_n = 80.0                     
# ----------------------------------------------------------------

## 3. Read noise files

In [29]:
# ------> Specify folder locations

rawdata_dir = '/home/manuel/Research Projects/GW Data analysis/GitHub/CCSNeHFGW_ResNetClass/Codes/'

# ------> Initialize lists

time_noise = []
strain_noise = []
F_noi = []

# ------> Scan noise files

os.chdir(rawdata_dir)

for file in glob.glob("strain_noise_" + detector + "*"):
    F_noi.append(file)

In [30]:
# ------> Load noise data 

result = re.search( detector + "_", F_noi[0])
ind_ini_n = result.span()[1]
result = re.search( ".txt", F_noi[0])
ind_end_n = result.span()[0]

print("Available noise segments (in GPS start time)")
for j in range(len(F_noi)):
    print("GPS =", F_noi[j][ind_ini_n:ind_end_n], "  |  Input option :", j)
print("")

nfile_i = input("======> Enter your option:")
nfile_i = int(nfile_i)
print("")

print("***** READING FILE", F_noi[nfile_i], " *****")

with open(F_noi[nfile_i]) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Data columns: {", ".join(row)}')
            line_count += 1
        else:
            time_noise.append(row[0])
            strain_noise.append(row[1])
            #print(f'\t{row[0]} works in the {row[1]}.')
            line_count += 1
    print(f'Processed {line_count} lines')

Available noise segments (in GPS start time)
GPS = 1257050112   |  Input option : 0
GPS = 1256783872   |  Input option : 1



======> Enter your option: 0



***** READING FILE strain_noise_V1_1257050112.txt  *****
Data columns: time, strain
Processed 16777217 lines


In [31]:
# ------> Convert lists to numpy arrays, clear memory from lists

time_n = np.array(time_noise, dtype='float64')
del(time_noise)
gc.collect()

strain_n = np.array(strain_noise, dtype='float64')
del(strain_noise)
gc.collect()

0

## 4. Injections preparation

In [32]:
# Compute sampling time

ts = time_n[1] - time_n[0]

# -------> JUST FOR DEBUGGING THE IMPLEMENTATION
#          Select a portion of the noise strain data

if reduce_segment:
    
    reduced_length = int(reduced_time_n/ts)     # Reduced noise arrays' length (in sample units)
    
    time_n = np.delete(time_n,np.arange(reduced_length, len(time_n), 1, dtype=int))
    strain_n = np.delete(strain_n,np.arange(reduced_length, len(strain_n), 1, dtype=int))
    
    #print("time_n vector:", time_n)
    print("**** length of time_n (in seconds):", len(time_n)*ts)
    print("**** length of time_n (in sample units):", len(time_n))
    print("**** length of strain_n (in sample units):", len(strain_n))

In [33]:
# ------> Backup original strain arrays
strain_n_new = np.copy(strain_n) # ---> Deep copy in numpy, to unbind arrays

# ------> Print useful input information

# Sampling frequency
fs =1/ts
print("Sampling frequency (in Hz): ", fs)

# Index of the strain noise sample (point) in which the first injection will be performed
N_lapse = int(dt_inj/(1./fs))
print("First noise sample to be injected with a waveform (without jitter): ", N_lapse)
print("Its correspondent strain value:", strain_n_new[N_lapse])

# Number of injections to be performed in the strain data
N_inj = int(len(strain_n_new) / N_lapse) - 1 
# Remark: -1 value in N_inj is just to avoid incomplete injection at the right edge.  
print("Number of injections to be performed: ", N_inj)

Sampling frequency (in Hz):  4096.0
First noise sample to be injected with a waveform (without jitter):  32768
Its correspondent strain value: -5.222492925612312e-20
Number of injections to be performed:  511


In [34]:
# ------> Load waveforms dictionaries

# Waveforms strain
with open('../Waveforms_mod/Phen/waveforms.pkl', 'rb') as fp:
    waveforms = pickle.load(fp)
    print('waveforms dictionary loaded successfully')

# Waveforms log data
with open('../Waveforms_mod/Phen/waveforms_log.pkl', 'rb') as fp:
    waveforms_log = pickle.load(fp)
    print('waveforms_log dictionary loaded successfully')
    
# Remark: How to access waveforms and log data
# waveforms["class c"][n]  ---> to extract n-th waveform of class c
# waveforms_log["class c"][n]  ---> to extract log data of n-th waveform of class c

waveforms dictionary loaded successfully
waveforms_log dictionary loaded successfully


In [35]:
# Checks
print(len(waveforms["class 1"]))
print(len(waveforms["class 2"]))

200
200


In [36]:
# ------> Select random waveforms for injections

# Predictable random numbers
np.random.seed(rdm_seed)

# Select N_inj random injections for the chosen class
inj_ind = np.random.choice(len(waveforms["class " + waveform_class]), N_inj)

# Print indexes of selected waveform for injection
print("Choosen waveforms: ", inj_ind)

Choosen waveforms:  [102 179  92  14 106  71 188  20 102 121  74  87 116  99 103 151 130 149
  52   1  87 157  37 129 191 187  20 160  57  21  88  48  58 169 187  14
 189 189 174 189  50 107  54  63 130  50 134  20  72 166  17 131  88  59
  13   8  89  52 129  83  91 110 187 198 171   7 174  34  80 163  49 103
 131   1 133  53 105   3  53 190 145  43 161 189  13  94  47  14 199 189
  39  81 110  52  23 153 187 123  40 156  14  44  64  88  70   8  87 128
 135  62 138  80 135 162 162  32 122   4  40  27 134  71  11 161  32  47
 150  61  36  98 171 103  34 192 100 174 130   0   4 141 102  26 136  14
  89  41 123 178  62  95  51  95 131 150 142 170  28  35  12 159  70 186
  85  27  65 169  44  61 184 133  27  27 107  43  83  29 189  74 127  91
 189 128 120  26 189 120 115   2 102 197 199 154 136  61 164  50 171 151
  58 117 159  95 179 112  61 185  51  11  38 129 130 112 100 112 183  80
 186 112   1 129  53  86 128 146 125 129  52 171 159 197 159  67 182 183
 122 144  37  23  68 115  97 19

## 5. Injections application

In [37]:
doplots_snr = 0 # Do plots in SNR calculation (checks)

# ------> Create array to locate injections by indexes

locate_inj = np.arange(1,N_inj+1)
locate_inj = N_lapse*locate_inj
#print(locate_inj[-1] + N_lapse)
#print(len(strain_n_new))

# ------> Define length of local noise segment for SNR calculation

epsilon = int(int(t_segment/ts)*0.5)
#print("Epsilon = ", epsilon)

# ------> Initialize list for log file

log_data = []

# ------> Make injections

# FIRST LOOP, j index: injection count variable
# -----------------------------------------------

for j in range(N_inj): # Apply N_inj injections
#for j in range(10): # Apply only a few injections
        
    print("Injection No.", j)
    print("----------------------")
    
    # ------> Load random waveform indexed by inj_ind array
    
    strain_wf = np.transpose(waveforms["class " + waveform_class][inj_ind[j]])[1]
    
    # ------> Load waveform log data
    
    slope = waveforms_log["class " + waveform_class][inj_ind[j]][0]
    f0 = waveforms_log["class " + waveform_class][inj_ind[j]][1]
    f1 = waveforms_log["class " + waveform_class][inj_ind[j]][2]
    wf_duration = waveforms_log["class " + waveform_class][inj_ind[j]][3]
    
    slope = float(slope)
    f0 = float(f0); f1 = float(f1)
    wf_duration = float(wf_duration)
    
    # ------> Injection location without jitter
    
    ind_noise = locate_inj[j] # Locate first point of the injection
    # print("Injection first point in the strain noise data (without jitter):", ind_noise)
    
    time_inj = (1./fs)*ind_noise
    print("Time injection, without jitter (in seconds):", time_inj)
    
    # ------> Jitter variable procedure
    
    # Define the jitter range in terms of sample units
    njitter_range_min = ind_noise + int(-jitter_lim/(1./fs))
    njitter_range_max = ind_noise + int(jitter_lim/(1./fs))
    njitter_range = np.arange(njitter_range_min, njitter_range_max + 1, 1, dtype=int)

    # Compute actual location of the injection in sample units
    njitter = np.random.choice(njitter_range, 1)[0]
    
    # Some checks
    #print("Jitter variable range (in sample units):", njitter_range)
    #print("Modified injection location (in sample units):", njitter)
    current_jitter = (njitter-ind_noise)*(1/fs)
    print("Current jitter (in seconds):", current_jitter)
    
    time_inj_jit = njitter*(1/fs)
    print("Time injection, with jitter (in seconds):", time_inj_jit)

    # ------> Update index injection locations
    
    locate_inj[j] = njitter 
    
    
    # SECOND LOOP, k index: Waveform's sample count variable
    # --------------------------------------------------------
    
    for k in range(len(strain_wf)): # Injection along waveform's ALL points
    #for k in range(3): # Injection along waveform's first three points
    #for k in reversed(range(len(strain_wf)-3,len(strain_wf),1)): # Injection along waveform's last three points
        
        #print("")
        #print("+++++ Point", k, " of the waveform")
        
        strain_waveform = strain_wf[k] 
        #print("------> strain waveform", strain_waveform)
        
        strain_noise = strain_n_new[locate_inj[j]+k]
        #print("------> strain BEFORE injection", strain_noise)
    
        # PERFORM THE INJECTION (k point in the waveform, locate_inj[j] + k point in the noise)
        strain_with_inj = strain_noise + strain_waveform
        strain_n_new[locate_inj[j]+k] = strain_with_inj
    
        #print("------> strain AFTER injection", strain_n_new[ind_noise+k])
        #print("------> strain AFTER injection in copy", strain_n[ind_noise+k])
    
    print("")
    #print("+++++ SNR calculation +++++")
    
    # ------> Define time segment for SNR calculation around the injection
    
    Llim_local_noise = ind_noise - epsilon
    Rlim_local_noise = ind_noise + len(strain_wf) + epsilon
    
    #print("Left limit local noise = ", Llim_local_noise)
    #print("Right limit local noise = ", Rlim_local_noise)
    
    #print("Strain noise length (in sample units):", len(strain_n_new))
    
    # Define strain noise segment
    strain_segment = strain_n_new[Llim_local_noise:Rlim_local_noise+1]
    
    #print("Strain noise segment length (in sample units):", len(strain_segment))
    
    # ------> Compute PSD
    
    # Size of Welch's segments
    nperseg = int(4*fs)
    
    # Double-sided power spectral density of raw strain data
    fpsd, psd = PSD(strain_segment,fs,nperseg,2)
    
    # Sort arrays, increasing frequencies
    ind = np.argsort(fpsd)
    fpsd = np.sort(fpsd)
    psd = psd[ind]
    
    # ------> Compute SNR values (with ScyPy)
    
    #SNR_value = SNR(strain_wf,time_wf,fpsd,psd,doplots_snr)
 
    #print("Injection SNR value (SciPy): ", SNR_value)
    
    # ------> Compute SNR values (with PyCBC)
    
    SNR_value = SNR_PyCBC(strain_wf, strain_segment, fs, doplots_snr)
    
    if np.isnan(SNR_value):
            
        print("!!!!!!!!!!!!!!!!! STOPPING INJECTION PROCEDURE, SNR VALUE IS \"NAN\" !!!!!!!!!!!!!!!!! ")
        break
    
    # ------> Print relevant log information
    
    print("G-mode slope =", slope)
    #print("G-mode start frequency , f0 = ", f0)
    #print("G-mode end frequency , f1 = ", f1)
    print("Waveform duration =", wf_duration)
    print("Injection SNR value (PyCBC): ", SNR_value)
        
    # ------> Save information in log_data list
    # Injection time, jitter, SNR value, waveform duration
    log_data.append([time_inj_jit, current_jitter, SNR_value, slope, f0, f1, wf_duration])
    
    print("****************************************************************")
    print("")        

Injection No. 0
----------------------
Time injection, without jitter (in seconds): 8.0
Current jitter (in seconds): 0.00390625
Time injection, with jitter (in seconds): 8.00390625

G-mode slope = 1131.0
Waveform duration = 0.8037109375
Injection SNR value (PyCBC):  43.11939180504554
****************************************************************

Injection No. 1
----------------------
Time injection, without jitter (in seconds): 16.0
Current jitter (in seconds): -0.0029296875
Time injection, with jitter (in seconds): 15.9970703125

G-mode slope = 990.0
Waveform duration = 0.849609375
Injection SNR value (PyCBC):  52.68671131340743
****************************************************************

Injection No. 2
----------------------
Time injection, without jitter (in seconds): 24.0
Current jitter (in seconds): 0.009033203125
Time injection, with jitter (in seconds): 24.009033203125

G-mode slope = 1171.0
Waveform duration = 0.92333984375
Injection SNR value (PyCBC):  56.124125593692

## 6. Check injections

In [38]:
%%script false --no-raise-error # WARNING: DEACTIVATED CELL

# ------> Check plot for strain before/after injection

# Select an injection
inj_point_1 = locate_inj[0]
inj_point_2 = locate_inj[-1]
#inj_point_2 = locate_inj[5]
min_index = inj_point_1-100
max_index = inj_point_2+len(strain_wf)+5000

# Subset of points
time_n_subset = time_n[min_index:max_index]
strain_n_subset = strain_n[min_index:max_index]
strain_n_new_subset = strain_n_new[min_index:max_index]

plt.figure(1, figsize=(10,7))
#plt.plot(time_n_subset,strain_n_subset, label='Before injection')
#plt.plot(time_n_subset,strain_n_new_subset, label='After injection')
plt.plot(time_n_subset,strain_n_subset-strain_n_new_subset, label='difference')
#plt.yscale('log')
plt.title("Strain difference before/after injection", fontsize=18)
plt.xlabel('post-bounce time [ms]', fontsize=16)
plt.ylabel('Diff h(t)', fontsize=16)
plt.grid()
#plt.legend()
plt.tight_layout()
plt.show()
plt.figure(1).clear()
gc.collect()

## 7. Save data

In [39]:
# ------> Log files, information of injections

# Pre-processed data folder
save_dir = '/home/manuel/Research Projects/GW Data analysis/GitHub/CCSNeHFGW_ResNetClass/Codes/Preprocessed_Data/'

# Create df_log dataframe
df_log = pd.DataFrame(log_data, columns=['Injection time [s]', 'jitter (seconds)', 'Waveform SNR', 'G-mode slope', 'Frequency f0 [Hz]', 'Frequency f1 [Hz]', 'Waveform duration [s]'])

# Export df_log dataframe to a csv file
df_log.to_csv(save_dir + "log_" + detector + "_" + F_noi[nfile_i][ind_ini_n:ind_end_n] + "_wfclass" + waveform_class +  '.dat', index=False)

# ------> Strain data with injections

# Create df_strain dataset
df_strain = pd.DataFrame({"time" : time_n, "strain" : strain_n_new})

# Export df_strain dataframe to a csv file
df_strain.to_csv(save_dir + "strain_" + detector + "_" + F_noi[nfile_i][ind_ini_n:ind_end_n] + "_injected_" + "wfclass" + waveform_class + ".txt", index=False)